In [4]:
import json
import time
import tweepy

import numpy as np

from confluent_kafka import Consumer, KafkaException, KafkaError, Producer
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

from utilities import *
from tweet_producer import *

In [5]:
with open('TWITTER_CREDENTIALS.JSON') as file:
    twitter_credentials = json.load(file)
with open('KARAFKA_CREDENTIALS.JSON') as file:
    karafka_credentials = json.load(file)

conf = {
            'bootstrap.servers': karafka_credentials["SERVERS"],
            'session.timeout.ms': 6000,
            'group.id': "%s-consumer" % karafka_credentials["USERNAME"],
            'default.topic.config': {'auto.offset.reset': 'smallest'},
            'security.protocol': 'SASL_PLAINTEXT',
            'sasl.mechanisms': 'SCRAM-SHA-256',
            'sasl.username': karafka_credentials["USERNAME"],
            'sasl.password': karafka_credentials["PASSWORD"],
            'api.version.fallback.ms': 0,
            'broker.version.fallback': '0.10.0'        
        }

conf_local = {
            'bootstrap.servers': 'localhost:9092',
            'session.timeout.ms': 6000,
            'default.topic.config': {'auto.offset.reset': 'smallest'}, 
            'queue.buffering.max.messages': 1000000, 
            'queue.buffering.max.ms' : 10,  
            'batch.num.messages': 10,   
            }

def twitter_setup():
    auth = OAuthHandler(twitter_credentials["KEY"], twitter_credentials["SECRET_KEY"])
    auth.set_access_token(twitter_credentials["ACCESS_TOKEN"], twitter_credentials["ACCESS_TOKEN_SECRET"])
    
    return auth

In [ ]:
auth = twitter_setup()
tweet_producer = TweetKafkaProducer(conf_local)
stream = Stream(auth, tweet_producer)
stream.sample(stall_warnings=True)